# 09. Computing coarse-grained molecular features

This notebook shows how to compute pairwise ditances, angles and dihedrals between CG beads given a CG mapping. The CG mapping used in this example is generated from [DSGPM](https://github.com/rochesterxugroup/DSGPM) model. 
You must need MDAnalysis and NetworkX in your working environment to run this example. 

In [1]:
import hoomd
import hoomd.htf as htf
import tensorflow as tf
import MDAnalysis as mda
import numpy as np

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import warnings
warnings.filterwarnings('ignore')

2 Physical GPUs, 2 Logical GPUs


In this example we read from a trajectory file to compute coarse-grained (CG) bond distances, angles and dihedrals. Here, we use two MDAnalysis universes with and without hydrogens as the mapping model used to compute the CG mappings (DSGPM model) only maps heavy atoms of a given molecule. Hence, we have to add the missing hydrogen atoms to the corresponding CG beads. 

## Read frames from the trajectory

In [2]:
universe = mda.Universe('ex9_segA.pdb','ex9_segA.trr')

In [3]:
avg_cgr = tf.keras.metrics.MeanTensor()
avg_cga = tf.keras.metrics.MeanTensor()
avg_cgd = tf.keras.metrics.MeanTensor()
directory = os.getcwd()
jfile = os.path.join(directory,'ex9_cgmap_segA.json')

#mda universe without H's
uxh = mda.Universe(os.path.join(directory,'ex9_segA_xH.pdb')) 
#mda universe with H's
uh = mda.Universe(os.path.join(directory,'ex9_segA.pdb')) 

for inputs, ts in htf.iter_from_trajectory(16, universe, r_cut=10, start=400, end=700):
    cg_fts = []
    r_tensor = []
    a_tensor = []
    d_tensor = []
    
    box = inputs[2]
    
    #get CG bead indices that make bonds, angles, dihedrals and
    #CG coordinates
    cg_fts = htf.compute_cg_graph(DSGPM=True,infile=jfile,group_atoms=True,
                                  u_no_H=uxh, u_H=uh)

    for i in range(len(cg_fts[0])):
        cg_r = htf.mol_bond_distance(CG = True, cg_positions = cg_fts[-1],
                                     b1=cg_fts[0][i][0],b2=cg_fts[0][i][1],box=box)
        r_tensor.append(cg_r)

    for j in range(len(cg_fts[1])):       
        cg_a = htf.mol_angle(CG= True, cg_positions=cg_fts[-1],
                             b1=cg_fts[1][j][0],b2=cg_fts[1][j][1],b3=cg_fts[1][j][2],box=box)
        a_tensor.append(cg_a)


    for k in range(len(cg_fts[2])):
        cg_d = htf.mol_dihedral(CG=True,cg_positions=cg_fts[-1],b1=cg_fts[2][k][0],
                                b2=cg_fts[2][k][1],b3=cg_fts[2][k][2],b4=cg_fts[2][k][3],box=box)
        d_tensor.append(cg_d)

    avg_cgr.update_state(r_tensor)
    avg_cga.update_state(a_tensor)
    avg_cgd.update_state(d_tensor)
        

100%|██████████| 801/801 [00:36<00:00, 21.85it/s] 


In [5]:
cgR = avg_cgr.result().numpy()
cgD = avg_cgd.result().numpy()*180./np.pi
cgA = avg_cga.result().numpy()*180./np.pi
print('CG pairwise distances:',cgR,'\n')
print('CG angles:',cgA,'\n')
print('CG dihedral angles:',cgD)

CG pairwise distances: [ 5.4447026  1.1312699  6.837509   2.9382832  2.4656553  4.441705
  3.1996844  4.215056   3.5845423  2.153629   7.9029675  3.8829584
  6.7588925  6.4774585  2.2552955  4.924933  15.14331  ] 

CG angles: [ 57.068886  75.223785  83.65739  113.90877   30.891739  61.174538
  40.556366  27.594133  50.535973 149.74693   46.743954  91.213806
  44.42919  157.15253   45.61488  121.533    140.93065   90.678955
  51.73324  156.72867 ] 

CG dihedral angles: [ 61.196335  177.25372     4.786071  111.41953   176.07283   133.15508
  84.994835  135.76688   147.1387      4.834333  168.7405    124.28169
 175.61646    21.146246  163.78929    32.634483    9.021207  175.17809
  10.565365    7.1954155]
